[tfrecord tutorial from TensorFlow](https://www.tensorflow.org/tutorials/load_data/tfrecord)


In [1]:
#import matplotlib.pyplot as plt
import numpy as np
#import tensorflow_datasets as tfds
import os
#from tensorflow_datasets.audio import nsynth
import tensorflow as tf
import json
from google.protobuf.json_format import MessageToJson

In [2]:
PATH_TRAIN = r"E:\nsynth233\2.3.3\train"
PATH_VALID = r"E:\nsynth233\2.3.3\valid"
PATH_TEST = r"E:\nsynth233\2.3.3\test"
#TFRECORD_FILES_TRAIN = os.listdir(PATH_TRAIN)
#TFRECORD_FILES_VALIDATION = os.listdir(PATH_VALID)
TFRECORD_FILES_TEST = os.listdir(PATH_TEST)


In [ ]:
nsynth._AUDIO_RATE
nsynth._DESCRIPTION

In [ ]:
nsynth._SPLITS

# Inspect single TFR example

## gansynth subset

In [ ]:


dataset = tf.data.TFRecordDataset(r"E:\nsynth233\2.3.3\train\nsynth-train.tfrecord-00000-of-00128")
for d in dataset.take(1):
    ex = tf.train.SequenceExample()
    ex2 = ex.ParseFromString(d.numpy())
    gan_ex = json.loads(MessageToJson(ex))
    #print(m['features']['feature'].keys())

In [ ]:
ex3 = ex.context.feature
ex3["audio"]

In [ ]:
print(gan_ex['context']['feature'].keys())

In [ ]:
gan_ex['context']['feature']['f0/hz']

In [ ]:
gan_ex['context']['feature']['f0/confidence']

In [ ]:
gan_ex['context']['feature']['loudness/db']

In [ ]:
gan_ex['context']['feature']['f0/midi']

In [ ]:
gan_ex['context']["feature"]["audio"]

## 2.3.3 full

In [ ]:
import tensorflow as tf
import json
from google.protobuf.json_format import MessageToJson

dataset = tf.data.TFRecordDataset(r"datasets_nsynth_full_2.3.3_nsynth-test (1).tfrecord-00000-of-00008")
for d in dataset.take(1):
    ex = tf.train.SequenceExample()
    ex2 = ex.ParseFromString(d.numpy())
    m = json.loads(MessageToJson(ex))
    #print(m['features']['feature'].keys())

In [ ]:
ex3 = ex.context.feature
ex3["audio"]

In [ ]:
print(m['context']['feature'].keys())

In [ ]:
m['context']['feature']['loudness/db']

In [ ]:
m['context']["feature"]["audio"]

# Read and Write TFR Files

take some random example to write

In [ ]:
files_to_write = []

In [ ]:
for raw_record in dataset.take(2):
    files_to_write.append(raw_record)
    example_write = tf.train.SequenceExample()
    write_to_disk = raw_record

#print(write_to_disk)

(other way to represent it)

1) add the tensor file to tf.data.Dataset (more could be added)
2) write the file to some filename (it should now be visible in your folder structure)

In [ ]:


filename_example = "nsynth-valid-nico.tfrecord-111-of-32"
for idx,file in  enumerate(files_to_write):
    dataset_write = tf.data.Dataset.from_tensors(file)
    writer = tf.data.experimental.TFRecordWriter(filename_example)
    writer.write(dataset_write)




In [ ]:
dataset_write = tf.data.Dataset.from_tensor_slices(files_to_write)
writer = tf.data.experimental.TFRecordWriter(filename_example )
writer.write(dataset_write)

In [ ]:

raw_dataset_read = tf.data.TFRecordDataset(filename_example)
print(raw_dataset_read)
for raw_record_nico in raw_dataset_read.take(2):
    read_from_disk = raw_record_nico
    #print(read_from_disk)
    example_read = tf.train.SequenceExample()
    print(example_read.ParseFromString(read_from_disk.numpy()))

In [ ]:
for raw_record in raw_dataset_read.take(2):

    # extract instrument family
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    example_JSON = json.loads(MessageToJson(example))
    instrument_family = example_JSON["context"]["feature"]['instrument/family']['int64List']['value']

    #['feature']['instrument/family']['int64List']['value']
    print(instrument_family)


In [ ]:

filename = f"nsynth-valid-nico.tfrecord-00-of-32"
dataset_write = tf.data.Dataset.from_tensor_slices(files_to_write)
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(dataset_write)

read the same file back in and check if it is really the same

In [ ]:
example_read.ParseFromString(read_from_disk.numpy()) == example_write.ParseFromString(write_to_disk.numpy())

so the two files represent exactly the same object !

# Sorting by instrument family

## Validation Files

In [ ]:
processed_samples_info = []
original_order_instr_fam = []
for group_index,data_group in enumerate(TFRECORD_FILES_VALIDATION):
  data_group = tf.data.TFRecordDataset(PATH_VALID + "/" + data_group)
  print(group_index)
  for raw_record in data_group:
    if group_index<=20:
      parsed_record = tf.train.SequenceExample()
      parsed_record.ParseFromString(raw_record.numpy())
      # convert raw uninformative string to JSON
      for_inst_fam = json.loads(MessageToJson(ex))
      instrument_family = for_inst_fam['context']['feature']['instrument/family']['int64List']['value']
      #instrument_type = parsed_record.features.feature['instrument_family_str'].bytes_list.value[0]
      original_order_instr_fam.append(instrument_family)
      processed_samples_info.append([group_index,raw_record,instrument_family,parsed_record])
      #print(instrument_type)
    else:
        print("finished")

In [ ]:
ex = parsed_record
m = json.loads(MessageToJson(ex))

print(m['context']['feature'].keys())

In [ ]:
print(m['context']['feature']['instrument/family']['int64List']['value'])

In [ ]:

unique_fam_values = set([fam[0] for fam in original_order_instr_fam])
print(unique_fam_values)
# looks like all validation samples belong to instr family 5, what about instr label ?

In [ ]:
instrument_families = ["bass", "brass", "flute", "guitar", "keyboard","mallet","organ","reed","string","synth_lead","vocal"]
sorted_by_instr_val = {
"bass": [],
"brass": [],
"flute" : [],
"guitar" : [],
"keyboard" : [],
"mallet" : [],
"organ" : [],
"reed" : [],
"string" : [],
"synth_lead" : [],
"vocal" : []
}



def sort_by_instrument(data, instr_dict):
    for sample in data:
      instr_fam = sample[2]
      print(instr_fam)
      instr_dict[instrument_families[int(instr_fam[0])]].append(sample)
    return instr_dict


sorted_by_instr_val  = sort_by_instrument(processed_samples_info, sorted_by_instr_val)

In [ ]:
sorted_by_instr_val["guitar"][3]

In [ ]:
guitar = sorted_by_instr_val["guitar"][3]
#guitar


In [ ]:
test_dict = {
    "bass": [],
    "brass": [],
    "flute" : [],
    "guitar" : [[0,"tensor","type","parsed"]],
    "keyboard" : [],
    "mallet" : [],
    "organ" : [],
    "reed" : [],
    "string" : [],
    "synth_lead" : [],
    "vocal" : []
}

test_guitar = test_dict["guitar"]

In [ ]:
test_guitar

## Train Files

In [ ]:
# same as last one (todo ?)
def sort_raw_records_by_instrument(data):
    instrument_families = ["bass", "brass", "flute", "guitar", "keyboard","mallet","organ","reed","string","synth_lead","vocal"]
    instr_dict_train = {
        "bass": [],
        "brass": [],
        "flute" : [],
        "guitar" : [],
        "keyboard" : [],
        "mallet" : [],
        "organ" : [],
        "reed" : [],
        "string" : [],
        "synth_lead" : [],
        "vocal" : []
    }
    for sample in data:
      instr_fam = sample[1]
      #print(instr_fam)
      instr_dict_train[instrument_families[int(instr_fam[0])]].append(sample[0])
    return instr_dict_train

def writeTFRTrainFiles(instr_dict_train,sort_round):
    for key,instr_values in instr_dict_train.items():
        if instr_values:
            filename_train = f"{str(sort_round)}-train-{str(key)}-nsynth-subset-train.tfrecord-{str(sort_round)}-label-{str(key)}"
            dataset_write = tf.data.Dataset.from_tensor_slices(instr_values)
            writer = tf.data.experimental.TFRecordWriter(filename_train)
            writer.write(dataset_write)
        else:
            print(key," has no instrument values for sort round ",sort_round)
    return None

#sorted_by_instr_train  = sort_raw_records_by_instrument(processed_samples_info_train, sorted_by_instr_train)

In [ ]:

upper = 20
lower = -1

#sort_round = 0,1
sort_round = 2
processed_samples_info_train = []

STARTPOSITION = 38

for source_group_index,file_in_name in enumerate(TFRECORD_FILES_TRAIN[STARTPOSITION:]):
  data_group_train = tf.data.TFRecordDataset(PATH_TRAIN + "/" + file_in_name)
  #original_order_instr_fam_train = []
  if lower < source_group_index < upper: # adjusts per round (>= act / <= act+20)
      print(source_group_index)
      for raw_record in data_group_train:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          # convert raw uninformative string to JSON and extract relevant features
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          #instrument_label = processed['context']['feature']['instrument/label']
          #f0_hz = processed['context']['feature']['f0/hz']
          #f0_confidence = processed['context']['feature']['f0/confidence']
          #loudness_decibel = processed['context']['feature']['loudness/db']
          #audio = processed['context']["feature"]["audio"]
          #instrument_type = parsed_record.features.feature['instrument_family_str'].bytes_list.value[0]
          #original_order_instr_fam_train.append((instrument_family,instrument_label))
          processed_samples_info_train.append([raw_record,instrument_family])
  else:
      # write files to disk and clean memory
      upper +=20
      lower +=20
      instr_dict_train = sort_raw_records_by_instrument(processed_samples_info_train)
      writeTFRTrainFiles(instr_dict_train, sort_round)
      del instr_dict_train
      del processed_samples_info_train
      print(f"round {sort_round} finished")
      sort_round += 1
      
      # continue
      for raw_record in data_group_train:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          processed_samples_info_train = []
          processed_samples_info_train.append((raw_record,instrument_family))

In [ ]:
ex = parsed_record
m = json.loads(MessageToJson(ex))

print(m['context']['feature'].keys())

In [ ]:
print(m['context']['feature']['instrument/family']['int64List']['value'])

In [ ]:

unique_fam_values = set([fam[0] for fam in original_order_instr_fam])
print(unique_fam_values)

## Test files

In [3]:
# same as last one (todo ?)
def sort_raw_records_by_instrument(data):
    instrument_families = ["bass", "brass", "flute", "guitar", "keyboard","mallet","organ","reed","string","synth_lead","vocal"]
    instr_dict_test = {
        "bass": [],
        "brass": [],
        "flute" : [],
        "guitar" : [],
        "keyboard" : [],
        "mallet" : [],
        "organ" : [],
        "reed" : [],
        "string" : [],
        "synth_lead" : [],
        "vocal" : []
    }
    for sample in data:
      instr_fam = sample[1]
      #print(instr_fam)
      instr_dict_test[instrument_families[int(instr_fam[0])]].append(sample[0])
    return instr_dict_test

def writeTFRTestFiles(instr_dict_test,sort_round):
    for key,instr_values in instr_dict_test.items():
        if instr_values:
            filename_test = f"{str(sort_round)}-test-{str(key)}-nsynth-subset-test.tfrecord-{str(sort_round)}-label-{str(key)}"
            dataset_write = tf.data.Dataset.from_tensor_slices(instr_values)
            writer = tf.data.experimental.TFRecordWriter(filename_test)
            writer.write(dataset_write)
        else:
            print(key," has no instrument values for sort round ",sort_round)
    return None

#sorted_by_instr_train  = sort_raw_records_by_instrument(processed_samples_info_train, sorted_by_instr_train)

In [ ]:

upper = 20
lower = -1


sort_round = 0
processed_samples_info_test = []

STARTPOSITION = 0

for source_group_index,file_in_name in enumerate(TFRECORD_FILES_TEST[STARTPOSITION:]):
  data_group_test = tf.data.TFRecordDataset(PATH_TEST + "/" + file_in_name)
  #original_order_instr_fam_train = []
  if lower < source_group_index < upper: # adjusts per round (>= act / <= act+20)
      print(source_group_index)
      for raw_record in data_group_test:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          # convert raw uninformative string to JSON and extract relevant features
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          #instrument_label = processed['context']['feature']['instrument/label']
          #f0_hz = processed['context']['feature']['f0/hz']
          #f0_confidence = processed['context']['feature']['f0/confidence']
          #loudness_decibel = processed['context']['feature']['loudness/db']
          #audio = processed['context']["feature"]["audio"]
          #instrument_type = parsed_record.features.feature['instrument_family_str'].bytes_list.value[0]
          #original_order_instr_fam_train.append((instrument_family,instrument_label))
          processed_samples_info_test.append([raw_record,instrument_family])
  else:
      # write files to disk and clean memory
      upper +=20
      lower +=20
      instr_dict_test = sort_raw_records_by_instrument(processed_samples_info_test)
      writeTFRTestFiles(instr_dict_test, sort_round)
      del instr_dict_test
      del processed_samples_info_test
      print(f"round {sort_round} finished")
      sort_round += 1

      # continue
      for raw_record in data_group_test:
          parsed_record = tf.train.SequenceExample()
          parsed_record.ParseFromString(raw_record.numpy())
          processed = json.loads(MessageToJson(parsed_record))
          instrument_family = processed['context']['feature']['instrument/family']['int64List']['value']
          processed_samples_info_test = []
          processed_samples_info_test.append((raw_record,instrument_family))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [1]:
mallet_path_train = r"E:\nsynthgan_generation_sorted\mallet\train"

In [22]:
import tensorflow as tf
import glob
my_batch_size = 126
filenames_full = glob.glob(r"E:\nsynthgan_generation_sorted\mallet\train\*.tfrecord-*")


In [9]:
!pip install ddsp


In [11]:
class DataProvider(object):
    """Base class for returning a dataset."""

    def __init__(self, sample_rate, frame_rate):
        """DataProvider constructor.
        Args:
          sample_rate: Sample rate of audio in the dataset.
          frame_rate: Frame rate of features in the dataset.
        """
        self._sample_rate = sample_rate
        self._frame_rate = frame_rate

    @property
    def sample_rate(self):
        """Return dataset sample rate, must be defined in the constructor."""
        return self._sample_rate

    @property
    def frame_rate(self):
        """Return dataset feature frame rate, must be defined in the constructor."""
        return self._frame_rate

    def get_dataset(self, shuffle):
        """A method that returns a tf.data.Dataset."""
        raise NotImplementedError

    def get_batch(self,
                  batch_size,
                  shuffle=True,
                  repeats=-1,
                  drop_remainder=True):
        """Read dataset.
        Args:
          batch_size: Size of batch.
          shuffle: Whether to shuffle the examples.
          repeats: Number of times to repeat dataset. -1 for endless repeats.
          drop_remainder: Whether the last batch should be dropped.
        Returns:
          A batched tf.data.Dataset.
        """
        dataset = self.get_dataset(shuffle)
        dataset = dataset.repeat(repeats)
        dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
        dataset = dataset.prefetch(buffer_size=_AUTOTUNE)
        return dataset

In [12]:
class TfdsProvider(DataProvider):
    """Base class for reading datasets from TensorFlow Datasets (TFDS)."""

    def __init__(self, name, split, data_dir, sample_rate, frame_rate):
        """TfdsProvider constructor.
        Args:
          name: TFDS dataset name (with optional config and version).
          split: Dataset split to use of the TFDS dataset.
          data_dir: The directory to read TFDS datasets from. Defaults to
            "~/tensorflow_datasets".
          sample_rate: Sample rate of audio in the dataset.
          frame_rate: Frame rate of features in the dataset.
        """
        self._name = name
        self._split = split
        self._data_dir = data_dir
        super().__init__(sample_rate, frame_rate)

    def get_dataset(self, shuffle=True):
        """Read dataset.
        Args:
          shuffle: Whether to shuffle the input files.
        Returns:
          dataset: A tf.data.Dataset that reads from TFDS.
        """
        return tfds.load(
            self._name,
            data_dir=self._data_dir,
            split=self._split,
            shuffle_files=shuffle,
            download=False)


In [13]:
class NSynthTfds(TfdsProvider):
    """Parses features in the TFDS NSynth dataset.
    If running on Cloud, it is recommended you set `data_dir` to
    'gs://tfds-data/datasets' to avoid unnecessary downloads.
    """

    def __init__(self,
                 name='nsynth/gansynth_subset.f0_and_loudness:2.3.0',
                 split='train',
                 data_dir='gs://tfds-data/datasets',
                 sample_rate=16000,
                 frame_rate=250,
                 include_note_labels=True):
        """TfdsProvider constructor.
        Args:
          name: TFDS dataset name (with optional config and version).
          split: Dataset split to use of the TFDS dataset.
          data_dir: The directory to read the prepared NSynth dataset from. Defaults
            to the public TFDS GCS bucket.
          sample_rate: Sample rate of audio in the dataset.
          frame_rate: Frame rate of features in the dataset.
          include_note_labels: Return dataset without note-level labels
            (pitch, instrument).
        """
        self._include_note_labels = include_note_labels
        if data_dir == 'gs://tfds-data/datasets':
            logging.warning(
                'Using public TFDS GCS bucket to load NSynth. If not running on '
                'GCP, this will be very slow, and it is recommended you prepare '
                'the dataset locally with TFDS and set the data_dir appropriately.')
        super().__init__(name, split, data_dir, sample_rate, frame_rate)

    def get_dataset(self, shuffle=True):
        """Returns dataset with slight restructuring of feature dictionary."""
        def preprocess_ex(ex):
            ex_out = {
                'audio':
                    ex['audio'],
                'f0_hz':
                    ex['f0']['hz'],
                'f0_confidence':
                    ex['f0']['confidence'],
                'loudness_db':
                    ex['loudness']['db'],
            }
            if self._include_note_labels:
                ex_out.update({
                    'pitch':
                        ex['pitch'],
                    'instrument_source':
                        ex['instrument']['source'],
                    'instrument_family':
                        ex['instrument']['family'],
                    'instrument':
                        ex['instrument']['label'],
                })
            return ex_out

        dataset = super().get_dataset(shuffle)
        dataset = dataset.map(preprocess_ex, num_parallel_calls=_AUTOTUNE)
        return dataset

In [51]:
    def preprocess_ex(ex):
        ex_out = {
            'audio':
                ex['audio'],
            'f0_hz':
                ex['f0']['hz'],
            'f0_confidence':
                ex['f0']['confidence'],
            'loudness_db':
                ex['loudness']['db'],
        }
        return ex_out


processed = json.loads(MessageToJson(parsed_record))
processed= processed['context']['feature']
#instrument_label = processed['context']['feature']['instrument/label']

In [55]:
import ddsp
import tensorflow_datasets as tfds
import json
from google.protobuf.json_format import MessageToJson
ds = tf.data.TFRecordDataset(
    filenames_full
).batch(
    my_batch_size
)
#data_provider.download_and_prepare()
dataset_gan = tf.data.Dataset.from_tensor_slices(filenames_full)
def read(inp):
    return tf.data.TFRecordDataset(inp)

file_content = dataset_gan.map(read)


data_group_000 = tf.data.TFRecordDataset(r'E:\\nsynthgan_generation_sorted\\mallet\\train\\3-train-mallet-nsynth-subset-train.tfrecord-3-label-mallet')
for raw_record in data_group_000.take(2):
    parsed_record = tf.train.SequenceExample()
    parsed_record.ParseFromString(raw_record.numpy())
    # convert raw uninformative string to JSON and extract relevant features
    processed = json.loads(MessageToJson(parsed_record))
    processed= processed['context']['feature']
    preprocess_ex(processed)

KeyboardInterrupt: 

In [49]:

tfds.load(
    name = r'E:/nsynthgan_generation_sorted/mallet/train/3-train-mallet-nsynth-subset-train.tfrecord-3-label-mallet',
    data_dir=filenames_full,
    split="train",
    shuffle_files=True,
    download=False)

ValueError: Parsing builder name string E:/nsynthgan_generation_sorted/mallet/train/3-train-mallet-nsynth-subset-train.tfrecord-3-label-mallet failed.
The builder name string must be of the following format:
  dataset_name[/config_name][:version][/kwargs]

  Where:

    * dataset_name and config_name are string following python variable naming.
    * version is of the form x.y.z where {x,y,z} can be any digit or *.
    * kwargs is a comma list separated of arguments and values to pass to
      builder.

  Examples:
    my_dataset
    my_dataset:1.2.*
    my_dataset/config1
    my_dataset/config1:1.*.*
    my_dataset/config1/arg1=val1,arg2=val2
    my_dataset/config1:1.2.3/right=True,foo=bar,rate=1.2


In [44]:
def parse_tfexample(record):
    return tf.io.parse_single_example(record,features_dict)
def features_dict():
    base_features = {
        'audio':
            tf.io.VarLenFeature(dtype=tf.float32),
        'f0_hz':
            tf.io.VarLenFeature(dtype=tf.float32),
        'f0_confidence':
            tf.io.VarLenFeature(dtype=tf.float32),
        'loudness_db':
            tf.io.VarLenFeature(dtype=tf.float32),
    }

In [46]:
for raw_record2 in ds.take(1):

    feature_dict_ex=parse_tfexample(raw_record2)
    parsed_record2 = tf.train.SequenceExample()
    feature_dict_ex=parse_tfexample(parsed_record2)
    parsed_record2.ParseFromString(raw_record2.numpy())
    # convert raw uninformative string to JSON and extract relevant features
    processed2 = json.loads(MessageToJson(parsed_record))

ValueError: Input serialized must be a scalar